Import Requirements

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import albumentations as A

Set the Hyperparameters

In [2]:
model_name = "efficientnet_b0"
weights = None #"imagenet"
input_dims = (32, 32, 3)
num_classes = 100
epochs = 25
batch_size = 4
shuffle_size = 16
learning_rate = 0.001
precision = "float32"
model_dir = "models"

Create the Models

In [3]:
models = {
    'efficientnet_b0': tf.keras.applications.EfficientNetB0,
    'efficientnet_b1': tf.keras.applications.EfficientNetB1,
    'efficientnet_b2': tf.keras.applications.EfficientNetB2,
    'efficientnet_b3': tf.keras.applications.EfficientNetB3,
    'efficientnet_b4': tf.keras.applications.EfficientNetB4,
    'efficientnet_b5': tf.keras.applications.EfficientNetB5,
    'efficientnet_b6': tf.keras.applications.EfficientNetB6,
    'efficientnet_b7': tf.keras.applications.EfficientNetB7
}
inputs = tf.keras.Input(shape=input_dims)
backbone = models[model_name](include_top=True,
                        weights=weights,
                        input_shape=input_dims,
                        classes=num_classes)(inputs)
backbone.trainable = True
# backbone = tf.keras.layers.Dense(num_classes, activation="softmax")(backbone)
model = tf.keras.Model(inputs=inputs, outputs=backbone, name=model_name)

In [4]:
model.summary()

Model: "efficientnet_b0"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 efficientnetb0 (Functional)  (None, 100)              4177671   
                                                                 
Total params: 4,177,671
Trainable params: 4,135,648
Non-trainable params: 42,023
_________________________________________________________________


Import test training data (MNIST)

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


Preprocessing

In [6]:
def augment_preprocess(image):
    image = np.array(image)
    transform = A.Compose([
        A.RandomCrop(width=input_dims[0], height=input_dims[1]),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.Resize(height=input_dims[0], width=input_dims[1]),
    ])
    transformed_image = transform(image=image)["image"]
    return transformed_image

def one_hot(indicies):
    return tf.one_hot(indicies, num_classes)[0]

In [7]:
x_train_ds = tf.data.Dataset.from_tensor_slices(x_train)

x_train_ds = x_train_ds.map(
    lambda x: tf.numpy_function(
        augment_preprocess,
        inp=[x],
        Tout=[tf.uint8]),
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    name="augment")

y_train_ds = tf.data.Dataset.from_tensor_slices(y_train)

y_train_ds = y_train_ds.map(
    lambda x: tf.numpy_function(
        one_hot,
        inp=[x],
        Tout=[tf.float32]),
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    name="one_hot")
train_dataset = tf.data.Dataset.zip((x_train_ds, y_train_ds)).shuffle(shuffle_size).batch(batch_size)

In [8]:
for x, y in train_dataset:
    #print(x)
    print(y)
    break

(<tf.Tensor: shape=(4, 100), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.,

Training Settings

In [13]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_func = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

Training

In [22]:
for epoch in range(epochs):
    print(epoch)
    for step, (x, y) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            out = model(x, training=True)
            loss = loss_func(y_true=y[0], y_pred=out)
            if precision == "mixed_float16":
                loss = optimizer.get_scaled_loss(loss)
        gradients = tape.gradient(
            target=loss,
            sources=model.trainable_variables)
        if precision == "mixed_float16":
            gradients = optimizer.get_unscaled_gradients(gradients)
        optimizer.apply_gradients(
            grads_and_vars=zip(gradients, model.trainable_variables))
        if step % 100 == 0:
            model.save_weights(f"{model_dir}/{model_name}_weights_{step}.h5")
        print("Epoch: {}, Step: {}, Loss: {}".format(epoch, step, loss.numpy()))

# tf.keras.models.save_model(model, f"{model_dir}/{model_name}")

0
Epoch: 0, Step: 0, Loss: 4.655250549316406
Epoch: 0, Step: 1, Loss: 4.176300048828125
Epoch: 0, Step: 2, Loss: 5.202820777893066
Epoch: 0, Step: 3, Loss: 4.749520778656006
Epoch: 0, Step: 4, Loss: 4.791799545288086
Epoch: 0, Step: 5, Loss: 5.968967437744141
Epoch: 0, Step: 6, Loss: 4.221220016479492
Epoch: 0, Step: 7, Loss: 5.706993103027344
Epoch: 0, Step: 8, Loss: 4.475464820861816
Epoch: 0, Step: 9, Loss: 5.993929862976074
Epoch: 0, Step: 10, Loss: 5.015353202819824
Epoch: 0, Step: 11, Loss: 5.6083984375
Epoch: 0, Step: 12, Loss: 6.529723644256592
Epoch: 0, Step: 13, Loss: 6.094003200531006
Epoch: 0, Step: 14, Loss: 6.47358512878418
Epoch: 0, Step: 15, Loss: 5.533027648925781
Epoch: 0, Step: 16, Loss: 4.177698612213135
Epoch: 0, Step: 17, Loss: 5.612213611602783
Epoch: 0, Step: 18, Loss: 5.180880069732666
Epoch: 0, Step: 19, Loss: 7.220242500305176
Epoch: 0, Step: 20, Loss: 6.132220268249512
Epoch: 0, Step: 21, Loss: 5.882503509521484
Epoch: 0, Step: 22, Loss: 6.198249816894531
Ep